In [1]:
from spotify import access_token,user_id, db
from spotify import df as dataframe

you did it, your first class!


In [2]:
import json
import urllib
import requests
import re
import pandas as pd
import datetime as dt

user_id = user_id
dataframe = dataframe
access_token = access_token
headers = {'Content-Type':'application/json','Authorization': 'Bearer {token}'.format(token=access_token)}
BASE_URL = 'https://api.spotify.com/v1/'


        
def scrape_top_100():
    #create new chrome session
    driver = webdriver.Chrome()
    #load the web page
    driver.get(url)
    #wait for the page to fully load
    driver.implicitly_wait(5)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    cols = soup.find_all('td')
    return cols

def df_prep():
    artists = [col.text for col in cols if col.attrs['data-bind'] == 'artist']
    albums = [col.text for col in cols if col.attrs['data-bind'] == 'album']
    songs = [col.text for col in cols if col.attrs['data-bind'] == 'title']
    
    track_dict = {'Artist':artists,'Album':albums,'Title':songs}
    df = pd.DataFrame(track_dict)
    replace_dict = {'featuring':'','ft.':'','ft':'','feat':'','feat.':'','&':''}
    df['Title'] = df['Title'].str.replace(r"\*+",'')
    df['Artist'] = df['Artist'].replace(replace_dict,regex=True)
    df['Artist'] = df['Artist'].str.replace(r"\(.*\)",'')
    df = df.reset_index()
    
    track_n_artist = df['Title'].str.lower()+ ' ' + df['Artist'].str.lower()
    track_n_artist = track_n_artist.str.replace(' ','+')
    return track_n_artist


def api(series):
    correct = []
    for i,s in enumerate(series):
        queryParams = f'?q={s}&type=track'
        r = requests.get('https://api.spotify.com/v1/search' + queryParams, headers=headers)
        res = r.json()
        for y in res['tracks']['items']:
            if y['artists'][0]['name'].lower()[:5] in df['Artist'].iloc[i].lower():
                if y['name'].lower()[:4] == df['Title'].iloc[i][:4].lower():
                    correct.append((i,y['name'],y['artists'][0]['name'],y['id']))
    return correct
            
def uri_matches(correct):
    df2 = pd.DataFrame(correct)
    df2 = df2.rename(columns={0:'index',1:'track',2:'artist',3:'uri'})
    df2 = df2.drop_duplicates(subset='index',keep='first')
    df2.set_index('index')
    df2 = df2.reset_index()
    return df2

def measure(df,df2):
    df_outer = pd.merge(df,df2,how='outer',on='index')
    df_inner = pd.merge(df,df2,how='inner',on='index')
    missing = (len(df_outer) - len(df_inner)) / len(df)
    miss_df = df.drop(df_inner.index,axis=0)
    return print(f'Missing {round(missing*100)} Spotify matches'),miss_df

def playlist_name():
    today = dt.datetime.today()
    date = today.strftime('%m-%d-%Y')
    weekday = today.strftime('%A')
    name = f'WWOZ {weekday} {date}'
    return name

def create_playlist():
    url = BASE_URL + 'users/' + user_id + '/playlists'
    reqbody = json.dumps({'name': playlist_name(),
               'description':'Playlist of WWOZ radio plays scraped from 100 most recent songs',
              'public': 'true'})
    r = requests.post(url = url, data = reqbody, headers=headers)
    db.InsertPlaylist(playlist_name())
    return r.json()['id']

def check_redundancies():
    uris = [x[0] for x in db.GetUris()]
    new = df2['uri'].tolist()
    good_to_go = [uri for uri in new if uri not in uris]
    return good_to_go

def add_tracks_to_spotify(spotify_playlist_id):
    uri_list = list(map(lambda uri: 'spotify:track:'+ uri, check_redundancies()))
    reqbody = json.dumps({'uris': uri_list })
    r = requests.post(f'https://api.spotify.com/v1/users/{user_id}/playlists/{spotify_playlst_id}/tracks', 
            data=reqbody,headers=headers)
    if r.status_code in [200, 201]:
        print('WOOOO!!!')
    try:
        # use check_redundancies() to mask df2 (our matched tracks w/ URIs) for SQL Insert
        playlist_id = db.GetPlaylistid()
        df3 = df2.loc[df2['uri'].isin(check_redundancies())] 
        df3.apply(lambda x: db.InsertTrack(x['artist'],x['track'],x['uri'],playlist_id),axis=1)
    except:
        print('no luck adding these new tracks to database')
        
#1.) ---- scrape top 100
cols = scrape_top_100()

#2.) ---- prep df into query params
track_n_artist = df_prep(cols)

#3.) ---- call_api // store uri's in df
df2 = uri_matches(api(track_n_artist))

#4.) ---- make playlist // add tracks
if len(check_redundancies()) != 0:
    add_tracks_to_spotify(create_playlist())
else:
    print('Nothing new to add')
    
#5.) ---- output missing entries / counts
measure(df,df2)[1]



<ipython-input-2-b4ff42ee3cb6>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Title'] = df['Title'].str.replace(r"\*+",'')
<ipython-input-2-b4ff42ee3cb6>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Artist'] = df['Artist'].str.replace(r"\(.*\)",'')


Missing 36 Spotify matches
WOOOO!!!


NameError: name 'add_tracks_to_db' is not defined

In [1]:
check_redundancies()


NameError: name 'check_redundancies' is not defined

In [ ]:
r.json()['id']

In [ ]:
  
#     def InsertPlaylist(self,name):
#         db = self.conn.cursor()
#         date = dt.datetime.today().strftime('%m-%d-%Y')
#         db.execute("INSERT INTO playlists (name,date) VALUES (?,?)",(name,date))
#         self.conn.commit()
    
#     def InsertTrack(self,artist,song,uri,playlist_id):
#         db = self.conn.cursor()
#         db.execute("INSERT INTO tracks (artist,song,uri,playlist_id) VALUES (?,?,?,?)",(artist,song,uri,playlist_id))
#         self.conn.commit()

#     def GetSecrets(self):
#         db = self.conn.cursor()
#         db.execute("SELECT CLIENT_ID,CLIENT_SECRET,USER_ID FROM secrets")
#         return db.fetchone()
    
#     def GetTokens(self):
#         db = self.conn.cursor()
#         db.execute("SELECT refresh_token, access_token FROM tokens")
#         return db.fetchone()
    
#     def UpdateToken(self,newToken):
#         db = self.conn.cursor()
#         #SQL UPDATE tokens.db SET VALUE of ACCESS_TOKEN = newToken
#         db.execute("UPDATE tokens SET access_token = ?", (newToken,))
#         conn.commit()

In [ ]:
missing.head()

In [ ]:
r = requests.get('https://api.spotify.com/v1/me',headers=headers)

In [ ]:

r.json()

In [ ]:

# correct = []

# for i,x in enumerate(df['Title']):
#     queryParams = '?query={}&type=track&market=US&offset=0&limit=50'.format(urllib.parse.quote(x))
#     r = requests.get('https://api.spotify.com/v1/search' + queryParams, headers=headers)
#     res = r.json()
#     correct_name = []
#     try:
#         for y in res['tracks']['items']:

#             if x == y['name']:
#                 for z in y['artists']:
#                     if z['name'].lower()[:5] in df['Artist'].iloc[i].lower():
#                         correct.append((i,z['name'],y['name']))
#     except:
#         continue
                    
                

# for i,x in enumerate(track_artist_or_single):
#     if x[2] == 1:
#         queryParams = '?q={}&type=album&market=US&limit=1'.format(urllib.parse.quote(x[0]))
#         r = requests.get('https://api.spotify.com/v1/search' + queryParams, headers=headers)
#         res = r.json()
#         correct.append((i,res['albums']['items'][0]['id'],res['albums']['items'][0]['name']))
#     else:
#         queryParams = '?q={}&type=track&market=US&limit=5'.format(urllib.parse.quote(x[0]))
#         r = requests.get('https://api.spotify.com/v1/search' + queryParams, headers=headers)
#         res = r.json()
    
#         top5results_bytrack = res['tracks']['items']
#         for track in top5results_bytrack:
#             print(track['artists'][0]['name'])
# #             if track['artists'][0]['name'].lower()[:5] == x[1].lower()[:5]:
# #                 correct.append((i,track['id'],track['name']))
#         break
    
#     print(top5results_bytrack[0]['name'])
    
#     artistList = df['Artist'].str.lower().tolist()


#     correct = [track['id'] for track in top5results_bytrack if track['artists'][0]['name'].lower() in artistList]
#     correct_songs.append(correct)
